In [3]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F13 '''
def F13(X):
    f = bn.F13()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.06 ,random_state=0 , l1_ratio= 0.931, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 34.88 , C = 624.11 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_13_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_13_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [7]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [8]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=861467, Mon Apr 13 13:31:42 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.087203203597275e+03 1.0e+00 1.20e-01  1e-01  1e-01 0:00.0
    2     26 5.093001992171921e+03 1.1e+00 1.17e-01  1e-01  1e-01 0:00.0
    3     39 5.089920364074901e+03 1.1e+00 1.17e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5099.71978498] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5094.85463979] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 2.350340621822034e+03 1.9e+00 7.43e-03  6e-03  7e-03 0:00.9
  200   2600 2.347777248259895e+03 1.8e+00 1.28e-04  9e-05  1e-04 0:01.8
  300   3900 2.347776376852519e+03 2.0e+00 3.09e-06  2e-06  2e-06 0:02.7
  400   5200 2.347776376511081e+03 2.2e+00 7.86e-08  4e-08  5e-08 0:03.6
  500   6500 2.347776376510955e+03 2.4e+00 2.81e-09  1e-09  2e-09 0:04.5
  504   6552 2.347776376510955e+03 2.4e+00 2.58e-09  1e-09  1e-09 0:04.5
termination on tolfun=1e-11 (Mon Apr 13 13:31:47 2020)
final/bestever f-value = 2.347776e+03 2.347776e+03
incumbent solution: [-0.10014823  0.07742639 -0.20082453  0.01033553 -0.18289013  0.16285211
  0.02745698  0.39473135 ...]
std deviations: [1.46988620e-09 1.24358317e-09 1.32391026e-09 1.37895972e-09
 1.27511234e-09 1.20724298e-09 1.31118376e-09 1.40688678e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=901465, Mon Apr 13 13:31:47 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.08208737153

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5082.08737154] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5082.08737154] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4697.65759814] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in functi

   66    858 -1.046868159410054e+09 1.7e+00 1.36e+02  1e+02  1e+02 0:00.8
termination on tolfacupx=1000.0 (Mon Apr 13 13:31:49 2020)
final/bestever f-value = -1.045668e+09 -1.046868e+09
incumbent solution: [  85.81504484   -7.37599279 -387.55548542 -166.89137047 -435.73454096
  115.08808817  -11.22653845  508.07057974 ...]
std deviations: [124.8690558  123.37143505 127.60952641 132.17828105 130.20566956
 121.70723075 129.75045435 134.40180371 ...]
Run : 1
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=885744, Mon Apr 13 13:31:49 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.090866714052012e+03 1.0e+00 1.17e-01  1e-01  1e-01 0:00.0
    2     26 5.086544670108472e+03 1.1e+00 1.13e-01  1e-01  1e-01 0:00.0
    3     39 5.081329918811648e+03 1.1e+00 1.15e-01  1e-01  1e-01 0:00.0


...

   64    832 -1.116790640729254e+09 1.7e+00 1.34e+02  1e+02  1e+02 0:00.7
termination on tolfacupx=1000.0 (Mon Apr 13 13:34:45 2020)
final/bestever f-value = -1.052660e+09 -1.116791e+09
incumbent solution: [  54.04841255  -12.87225048 -452.22569159 -185.75448138 -589.25869925
   41.37118638   19.12641759  557.71883533 ...]
std deviations: [125.66872225 126.05207969 128.60729754 128.33941429 129.38627398
 129.06261181 123.50183816 128.61958382 ...]
Run : 29
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=876340, Mon Apr 13 13:34:45 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.091160128956363e+03 1.0e+00 1.21e-01  1e-01  1e-01 0:00.0
    2     26 5.091170060216809e+03 1.1e+00 1.21e-01  1e-01  1e-01 0:00.0
    3     39 5.091152045107895e+03 1.1e+00 1.23e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5091.39315441] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5091.21396065] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 2.361277150932543e+03 1.7e+00 1.19e-02  1e-02  1e-02 0:00.9
  200   2600 2.347778580145131e+03 1.9e+00 2.03e-04  1e-04  2e-04 0:01.8
  300   3900 2.347776376685631e+03 2.2e+00 1.97e-06  1e-06  1e-06 0:02.7
  400   5200 2.347776376510975e+03 2.3e+00 3.10e-08  1e-08  2e-08 0:03.6
  479   6227 2.347776376510955e+03 2.7e+00 3.12e-09  1e-09  2e-09 0:04.3
termination on tolfun=1e-11 (Mon Apr 13 13:34:50 2020)
final/bestever f-value = 2.347776e+03 2.347776e+03
incumbent solution: [-0.10014824  0.07742639 -0.20082453  0.01033553 -0.18289013  0.16285211
  0.02745699  0.39473135 ...]
std deviations: [1.78478878e-09 1.58909936e-09 1.65396446e-09 1.88451381e-09
 1.24021545e-09 1.51160316e-09 1.71109641e-09 1.58911221e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=918088, Mon Apr 13 13:34:50 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 5.082087371538836e+03 1.0e+00 1.22e-01  1e-01  1e-01 0:00.0
termination on tolfun=1e-1

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5082.08737154] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5082.087372, sigma=1.22e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5082.087372, sigma=1.22e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

   77   1001 -1.301149181426373e+09 1.8e+00 1.31e+02  1e+02  1e+02 0:00.9
termination on tolfacupx=1000.0 (Mon Apr 13 13:34:51 2020)
final/bestever f-value = -1.275189e+09 -1.301149e+09
incumbent solution: [-273.74376022  -36.11525163 -463.70621594 -250.92205756 -515.53724498
  176.79891053 -159.20456488  183.20386343 ...]
std deviations: [124.52065365 117.77847539 125.88062577 129.11167883 119.87649749
 124.62623421 131.00734497 119.63580431 ...]


In [9]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F13_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F13(X_Values.iloc[i,:25])
    SVM_Fun [i] = F13(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F13(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2205.7497941]), count=array([1])) 70.30086650772016
SVM
ModeResult(mode=array([2071.91166911]), count=array([1])) 84.7764410903182
ELN
ModeResult(mode=array([277141.12752504]), count=array([1])) 451178.21098140534
